In [1]:
import geopandas as gpd
import pandas as pd

,GEO2_MX,sum_income,total_pop,unrel_ppl,perc_urban,sum_num_intmig
0,1001,22382867663,849101.0,0.018720,0.892990,42055.0
1,1002,3998425640,45951.0,0.006768,0.269853,4017.0
2,1003,3045860068,53142.0,0.011385,0.493884,11992.0
3,1004,3449277596,14302.0,0.000839,0.367221,762.0
4,1005,3028539049,100150.0,0.019251,0.669675,7551.0


In [76]:
match = pd.read_csv("./gB_IPUMS_match.csv")
match = match[['shapeID', 'MUNI2015']]
ref_dict = dict(zip(match['MUNI2015'], match['shapeID']))
match.head()

,shapeID,MUNI2015
0,MEX-ADM2-1590546715-B1,1002
1,MEX-ADM2-1590546715-B2,1011
2,MEX-ADM2-1590546715-B3,1007
3,MEX-ADM2-1590546715-B4,1008
4,MEX-ADM2-1590546715-B5,1005


In [77]:
df = pd.read_csv("./mexico2010.csv")
df = df[['GEO2_MX', 'sum_income', 'total_pop', 'unrel_ppl', 'perc_urban', 'sum_num_intmig']]
df['GEO2_MX'] = df['GEO2_MX'].astype(str).str.replace("484", "").astype(int).map(ref_dict)
df.head()

,GEO2_MX,sum_income,total_pop,unrel_ppl,perc_urban,sum_num_intmig
0,MEX-ADM2-1590546715-B7,22382867663,849101.0,0.018720,0.892990,42055.0
1,MEX-ADM2-1590546715-B1,3998425640,45951.0,0.006768,0.269853,4017.0
2,MEX-ADM2-1590546715-B10,3045860068,53142.0,0.011385,0.493884,11992.0
3,MEX-ADM2-1590546715-B11,3449277596,14302.0,0.000839,0.367221,762.0
4,MEX-ADM2-1590546715-B5,3028539049,100150.0,0.019251,0.669675,7551.0


In [49]:
gdf = gpd.read_file("./MEX/MEX_ADM2_fixedInternalTopology.shp")
gdf.head()

,shapeName,shapeISO,shapeID,shapeGroup,shapeType,num_flt,geometry
0,Asientos,None,MEX-ADM2-1590546715-B1,MEX,ADM2,0.0,"POLYGON ((-101.99941 22.21951, -101.99940 22.2..."
1,San Francisco de los Romo,None,MEX-ADM2-1590546715-B2,MEX,ADM2,0.0,"POLYGON ((-102.27355 22.08805, -102.27320 22.0..."
2,Rincón de Romos,None,MEX-ADM2-1590546715-B3,MEX,ADM2,0.0,"POLYGON ((-102.18902 22.36244, -102.18951 22.3..."
3,San José de Gracia,None,MEX-ADM2-1590546715-B4,MEX,ADM2,0.0,"POLYGON ((-102.35385 22.12064, -102.35426 22.1..."
4,Jesús María,None,MEX-ADM2-1590546715-B5,MEX,ADM2,0.0,"POLYGON ((-102.32431 21.93627, -102.32438 21.9..."


In [78]:
# # add NEIGHBORS column
# gdf["NEIGHBORS"] = None  
# i = 0
# for index, country in gdf.iterrows():   

#     # get 'not disjoint' countries
#     neighbors = gdf[~gdf.geometry.disjoint(country.geometry)].shapeID.tolist()

#     # remove own name of the country from the list
#     neighbors = [ name for name in neighbors if country.shapeID != name ]

#     # add names of neighbors as NEIGHBORS value
#     gdf.at[index, "NEIGHBORS"] = ", ".join(neighbors)

#     i += 1

#     print(i)

#     # if i > 5:
#     #     break

In [62]:
def get_edge_indices(cur_id, neighbors):
    '''
    Get the edge indices between all of the boxes in a municipality
    '''
    # Initialize with self connections
    edge_indices = [[cur_id, cur_id]]

    for n in neighbors:
        edge_indices.append([cur_id, n])
        edge_indices.append([n, cur_id])

    return edge_indices

# print(len(all_ims))

In [165]:
import numpy as np
import torch

In [316]:
def get_x(shapeID, NEIGHBORS, dta):
    to_return = []
    to_return.append(dta[dta['GEO2_MX'] == shapeID].drop(['GEO2_MX', 'sum_num_intmig'], axis = 1).values[0])
    for i in NEIGHBORS:
        # print(i)
        if len(dta[dta['GEO2_MX'] == i]['sum_num_intmig'].values) != 0:
            to_return.append(dta[dta['GEO2_MX'] == i].drop(['GEO2_MX', 'sum_num_intmig'], axis = 1).values[0])
        else:
            to_return.append(np.array([0,0,0,0]))

    to_return = [i.tolist() for i in to_return]
    return np.array(to_return)


def get_edge_indices(cur_id, neighbors):
    '''
    Get the edge indices between all of the boxes in a municipality
    '''

    # print(cur_id, neighbors)

    edge_dict = {}
    edge_dict[cur_id] = 0
    for i in range(0, len(neighbors)):
        edge_dict[neighbors[i]] = i
    # print(edge_dict)

    # # Initialize with self connections
    edge_indices = [[edge_dict[cur_id], edge_dict[cur_id]]]

    for n in neighbors:
        edge_indices.append([edge_dict[cur_id], edge_dict[n]])
        edge_indices.append([edge_dict[n], edge_dict[cur_id]])

    # print(edge_indices)

    return edge_indices

In [317]:
def make_adj_list(target_id, neighbors_df, degrees):

    # print("TARGET: ", target_id)
    final_list = []
    target_neighbors = neighbors_df[neighbors_df['shapeID'] == target_id]['NEIGHBORS'].str.split(", ").values.tolist()[0]
    final_list.append(target_neighbors)
    for n in target_neighbors:
        cur_neighbors = neighbors_df[neighbors_df['shapeID'] == n]['NEIGHBORS'].str.split(", ").values.tolist()[0]
        final_list.append(cur_neighbors)

    unique_list = list(set([item for sublist in final_list for item in sublist]))
    id_dict = dict(zip(unique_list, [i for i in range(len(unique_list))]))

    max_list = 0
    for i in final_list:
        if len(i) > max_list:
            max_list = len(i)


    to_fill = np.full((len(final_list), max_list), -99)

    for i in range(0, len(final_list)):
        for j in range(0, len(final_list[i])):
            to_fill[i][j] = id_dict[final_list[i][j]]

    return to_fill



In [318]:
# gdf.head()

In [323]:
DEGREES = 1

all_x, all_adj_list = [], []

for col, row in gdf[0:5].iterrows():

    try:

        cur_adj_list = make_adj_list(row.shapeID, gdf, DEGREES)
        all_adj_list.append(cur_adj_list)

        cur_x = get_x(row.shapeID, row.NEIGHBORS.split(", "), df)
        all_x.append(cur_x)

    except:
        pass


In [324]:
all_x

[array([[3.99842564e+09, 4.59510000e+04, 6.76807904e-03, 2.69852669e-01],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [2.23828677e+10, 8.49101000e+05, 1.87197989e-02, 8.92990351e-01],
        [6.53055032e+09, 8.89420000e+04, 3.57536372e-03, 7.26271053e-01],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [4.58860751e+09, 3.12040000e+04, 3.52518908e-03, 5.46276118e-01],
        [1.99323171e+09, 1.11510000e+04, 1.34517084e-03, 4.98520312e-01],
        [2.88038142e+09, 2.88330000e+04, 6.58967156e-03, 5.00364166e-01]]),
 array([[2.55247238e+09, 7.16400000e+03, 0.00000000e+00, 5.63930765e-01],
        [2.07262582e+09, 2.00480000e+04, 0.00000000e+00, 4.16749800e-01],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [6.53055032e+09, 8.89420000e+04, 3.57536372e-03, 7.26271053e-01],
        [3.44927760e+09, 1.43020000e

In [ ]:
def one_degree_hotter(adj_list, already_visited):

    # new_adj_list = torch.ones(adj_list.shape[0], torch.max(adj_list) * torch.max(adj_list))

    for i in range(0, adj_list.shape[0]): 
        cur_nodes = adj_list[i]
        cur_stack = torch.tensor([-99]) 
        for j in cur_nodes:
            if j != -99:
                nodes_to_add_in = adj_list[j]
                for n in nodes_to_add_in:
                    if n not in cur_stack and n not in cur_nodes and n != i and n not in already_visited[i]:
                        cur_stack = torch.cat((cur_stack, n.unsqueeze(0)))
        ready_to_stack = torch.nn.functional.pad(cur_stack, (0, adj_list.shape[0] - cur_stack.shape[0]), value = -99).unsqueeze(0)
        try:
            final_stack = torch.cat((final_stack, ready_to_stack), dim = 0)
        except:
            final_stack = ready_to_stack

    return final_stack




class Sage(torch.nn.Module):

    def __init__(self, embed_dim, feature_dim, num_classes, K): 
        super(Sage, self).__init__()
        '''weights is of shape [embed_dim (arbitrary), feature_dim * 2]'''
        self.w2 = torch.nn.Parameter(torch.rand(num_classes, embed_dim), requires_grad = True)
        self.relu = torch.nn.ReLU()
        self.K = K

        self.params  = torch.nn.ParameterDict({})

        for i in range(K):
            if i == 0:
                feature_dim = feature_dim * 2
            self.params[str(i)] = torch.nn.Parameter(torch.rand(embed_dim, feature_dim), requires_grad = True)
            feature_dim = embed_dim * 2



    def forward(self, x, adj_list):
        
        already_visited = adj_list

        for degree in range(self.K):

            if degree == 0:

                for node in adj_list:
                    cur_neigh_feats = torch.mean(torch.index_select(x, 0, node[node >= 0]), dim = 0).unsqueeze(0)
                    try:
                        neigh_feats = torch.cat((neigh_feats, cur_neigh_feats), dim = 0)
                    except:
                        neigh_feats = cur_neigh_feats

                x = torch.cat((x, neigh_feats), dim = 1)
                x = torch.mm(self.params[str(degree)], x.t())
                x = self.relu(x).t()


            else:

                adj_list = one_degree_hotter(adj_list, already_visited)
                already_visited = torch.cat((already_visited, adj_list), dim = 1)

                for node in adj_list:
                    cur_neigh_feats = torch.mean(torch.index_select(x, 0, node[node >= 0]), dim = 0).unsqueeze(0)
                    try:
                        neigh_feats = torch.cat((neigh_feats, cur_neigh_feats), dim = 0)
                    except:
                        neigh_feats = cur_neigh_feats

                neigh_feats[neigh_feats != neigh_feats] = 0

                x = torch.cat((x, neigh_feats), dim = 1)
                x = torch.mm(self.params[str(degree)], x.t())
                x = self.relu(x).t()

            del neigh_feats

            # print(x.shape)

        # out = torch.mm(self.w2, x).t()

        return x




class sage_net(torch.nn.Module):

    def __init__(self, embed_dim, feature_dim, num_classes, K): 
        super(sage_net, self).__init__()
        self.s1 = Sage(embed_dim, feature_dim, num_classes, K)
        self.s2 = Sage(32, embed_dim, num_classes, K)
        # self.fc = torch.nn.Linear()

    def forward(self, x):

        x, adj_list = x[0], x[1]

        out = self.s1(x, adj_list)
        out = self.s2(out, adj_list)
        # print(out.shape)
        out = torch.mean(out)

        # print(out)

        return out